<a href="https://colab.research.google.com/github/jfoley40/ai-productivity-planner/blob/main/Python_Script_to_Create_ZIP_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile
import io

# Content of pod prep.html (taken from your uploaded file)
# For brevity, I'm representing this as a variable.
# In a real script, you would paste the full HTML content here,
# or read it from the actual 'pod prep.html' file if it's in the same directory.
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Prep Checklist & Idea Validator</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f7fafc; /* Light gray background */
        }
        /* Styles for completed tasks */
        .task-item.completed .task-text {
            text-decoration: line-through;
            color: #a0aec0; /* Gray out text */
        }
        .task-item.completed .checkbox-icon {
            background-color: #4299e1; /* Blue background for checkbox */
            border-color: #4299e1;
        }
        .task-item.completed .checkbox-icon svg {
            display: block; /* Show checkmark */
        }
        /* Button active state */
        .action-btn {
            transition: background-color 0.2s ease-in-out, transform 0.1s ease-in-out;
        }
        .action-btn:active {
            transform: translateY(1px); /* Slight press effect */
        }
        /* Custom scrollbar styles */
        ::-webkit-scrollbar { width: 8px; height: 8px; }
        ::-webkit-scrollbar-track { background: #edf2f7; border-radius: 10px; }
        ::-webkit-scrollbar-thumb { background: #a0aec0; border-radius: 10px; }
        ::-webkit-scrollbar-thumb:hover { background: #718096; }

        /* Message box for notifications */
        .message-box {
            position: fixed; bottom: 20px; left: 50%;
            transform: translateX(-50%) translateY(20px); /* Start off-screen */
            padding: 14px 22px; border-radius: 8px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.15);
            font-size: 0.95rem; font-weight: 500;
            z-index: 1000; opacity: 0; visibility: hidden;
            transition: opacity 0.3s ease, visibility 0.3s ease, transform 0.3s ease;
        }
        .message-box.show { opacity: 1; visibility: visible; transform: translateX(-50%) translateY(0); }
        .message-box.success { background-color: #38a169; color: white; } /* Green for success */
        .message-box.error { background-color: #e53e3e; color: white; } /* Red for error */

        /* Styling for content cards */
        .content-card {
            background-color: white; border-radius: 12px;
            box-shadow: 0 8px 16px rgba(0,0,0,0.05);
            transition: box-shadow 0.3s ease;
        }
        .content-card:hover { box-shadow: 0 12px 24px rgba(0,0,0,0.08); }

        /* Input field focus styles */
        .input-field {
            border-color: #e2e8f0;
            transition: border-color 0.2s ease, box-shadow 0.2s ease;
        }
        .input-field:focus {
            border-color: #3b82f6; /* Blue border on focus */
            box-shadow: 0 0 0 3px rgba(59, 130, 246, 0.2);
            outline: none;
        }
        /* Instructions card styling */
        .instructions-card {
            background-color: #e0f2fe; /* Light blue background */
            border-left: 4px solid #0ea5e9; /* Blue left border */
            border-radius: 8px; padding: 1.5rem; margin-bottom: 2.5rem;
        }
        .instructions-card h3 { color: #0369a1; font-weight: 600; margin-bottom: 0.75rem; }
        .instructions-card p, .instructions-card li { color: #075985; font-size: 0.95rem; line-height: 1.6; }
        .instructions-card ul { list-style-type: disc; padding-left: 1.5rem; margin-top: 0.5rem; }

        /* Print-specific styles */
        @media print {
            body {
                background-color: white !important;
                font-size: 12pt !important;
                color: black !important;
            }
            .container {
                max-width: 100% !important;
                padding: 0 !important;
                margin: 0 !important;
                width: 100% !important;
            }
            /* Hide elements not relevant for printing */
            header, .instructions-card, footer, form, .action-btn, .delete-btn, #export-buttons, #message-box, #clear-checklist-btn, #save-idea-btn, #clear-idea-btn  {
                display: none !important;
            }
            .content-card {
                box-shadow: none !important;
                border: 1px solid #ccc !important;
                margin-bottom: 20px !important;
                padding: 15px !important;
                page-break-inside: avoid !important; /* Try to keep cards on one page */
            }
            .grid {
                grid-template-columns: 1fr !important; /* Stack sections in print */
            }
            #task-list, .idea-question textarea {
                max-height: none !important; /* Show all content */
                overflow-y: visible !important;
            }
            .task-item {
                border-bottom: 1px solid #eee !important;
                padding-bottom: 5px !important;
                margin-bottom: 5px !important;
            }
            /* Style checkbox for print */
            .task-item .checkbox-icon {
                border: 1px solid #666 !important;
                background-color: white !important;
                min-width: 16px; /* Ensure checkbox is visible */
                min-height: 16px;
            }
            .task-item.completed .checkbox-icon {
                 background-color: #ccc !important; /* Show completed state */
            }
            .task-item.completed .checkbox-icon svg {
                filter: invert(100%); /* Make checkmark visible on gray bg */
                display: block !important;
            }
            .task-item .task-text {
                color: black !important;
            }
            .task-item.completed .task-text {
                text-decoration: line-through !important;
                color: #555 !important;
            }
            .idea-question label {
                font-weight: bold !important;
                margin-bottom: 0.25rem !important;
                display: block !important;
            }
            .idea-question textarea {
                border: none !important;
                padding: 0 !important;
                width: 100% !important;
                background-color: transparent !important;
                color: black !important;
                -webkit-print-color-adjust: exact !important; /* Ensure text color is printed */
                 print-color-adjust: exact !important;
                 resize: none !important;
            }
             h1, h2, h3 {
                color: black !important;
                page-break-after: avoid !important;
            }
        }
    </style>
</head>
<body class="antialiased text-slate-700">
    <div class="container mx-auto p-4 sm:p-6 md:p-8 max-w-5xl">
        <header class="mb-10 text-center">
            <h1 class="text-4xl md:text-5xl font-bold text-slate-800">Project Power Prep</h1>
            <p class="text-base text-slate-500 mt-2">Streamline your creation process from idea to pre-launch.</p>
        </header>

        <section class="instructions-card">
            <h2 class="text-xl font-semibold text-sky-700 mb-4">How to Use This Tool</h2>
            <div>
                <h3 class="text-lg">Pre-Launch Checklist:</h3>
                <ul class="space-y-1">
                    <li><strong>Add Tasks:</strong> Type a task into the input field (e.g., "Finalize script") and click "Add".</li>
                    <li><strong>Mark Complete:</strong> Click the checkbox next to a task to mark it as done.</li>
                    <li><strong>Delete Task:</strong> Click the "✕" icon next to a task to remove it.</li>
                    <li><strong>Clear All:</strong> Use the "Clear Entire Checklist" button to remove all tasks (confirm before deleting).</li>
                    <li><strong>Data Storage:</strong> Your checklist is automatically saved in your browser's local storage.</li>
                </ul>
            </div>
            <div class="mt-4">
                <h3 class="text-lg">Idea Validator:</h3>
                <ul class="space-y-1">
                    <li><strong>Enter Idea:</strong> Type your project title or core concept into the "Title / Core Idea" field.</li>
                    <li><strong>Refine with Questions:</strong> Answer the refinement questions to develop your idea.</li>
                    <li><strong>Save Notes:</strong> Click "Save Idea & Notes" to store your title and notes.</li>
                    <li><strong>Clear Fields:</strong> Click "Clear Current Fields" to empty the input fields (saved data remains until you save again).</li>
                    <li><strong>Data Persistence:</strong> Your idea and notes are saved in your browser.</li>
                </ul>
            </div>
            <div class="mt-4">
                <h3 class="text-lg">Exporting Your Data:</h3>
                <ul class="space-y-1">
                    <li><strong>Print to PDF:</strong> Click "Prepare for Print / PDF". Your browser's print dialog will open. Choose "Save as PDF".</li>
                    <li><strong>Copy for Docs:</strong> Click "Copy Content for Docs". Checklist and idea notes will be copied to your clipboard.</li>
                </ul>
            </div>
             <p class="mt-4 text-sm text-sky-600"><strong>Tip:</strong> Use this tool consistently for each new project to maximize its benefits!</p>
        </section>

        <section id="export-buttons" class="mb-8 p-4 bg-slate-100 rounded-lg shadow">
            <h2 class="text-xl font-semibold text-slate-700 mb-3 text-center">Export Your Content</h2>
            <div class="flex flex-col sm:flex-row justify-center items-center gap-4">
                <button id="print-pdf-btn" class="action-btn w-full sm:w-auto bg-blue-500 hover:bg-blue-600 text-white py-2 px-4 rounded-lg font-medium flex items-center justify-center">
                    <svg xmlns="http://www.w3.org/2000/svg" class="h-5 w-5 mr-2" viewBox="0 0 20 20" fill="currentColor"><path fill-rule="evenodd" d="M5 4v3H4a2 2 0 00-2 2v6a2 2 0 002 2h12a2 2 0 002-2V9a2 2 0 00-2-2h-1V4a2 2 0 00-2-2H7a2 2 0 00-2 2zm8 0H7v3h6V4zm0 8H7v4h6v-4z" clip-rule="evenodd" /></svg>
                    Prepare for Print / PDF
                </button>
                <button id="copy-content-btn" class="action-btn w-full sm:w-auto bg-green-500 hover:bg-green-600 text-white py-2 px-4 rounded-lg font-medium flex items-center justify-center">
                     <svg xmlns="http://www.w3.org/2000/svg" class="h-5 w-5 mr-2" viewBox="0 0 20 20" fill="currentColor"><path d="M8 3a1 1 0 011-1h2a1 1 0 110 2H9a1 1 0 01-1-1z" /><path d="M6 3a2 2 0 00-2 2v11a2 2 0 002 2h8a2 2 0 002-2V5a2 2 0 00-2-2 3 3 0 01-3 3H9a3 3 0 01-3-3z" /></svg>
                    Copy Content for Docs
                </button>
            </div>
        </section>

        <div class="grid grid-cols-1 lg:grid-cols-2 gap-8">
            <section class="content-card p-6 md:p-8">
                <h2 class="text-2xl font-semibold mb-5 text-sky-600">Pre-Launch Checklist</h2>
                <form id="task-form" class="flex mb-5">
                    <input type="text" id="new-task-input" placeholder="Add a new prep task (e.g., Outline content)" class="input-field flex-grow p-3 border rounded-l-lg focus:ring-sky-500 focus:border-sky-500" required>
                    <button type="submit" class="action-btn bg-sky-500 hover:bg-sky-600 text-white p-3 rounded-r-lg font-medium">Add</button>
                </form>
                <ul id="task-list" class="space-y-3 max-h-[30rem] overflow-y-auto pr-2">
                    </ul>
                <button id="clear-checklist-btn" class="action-btn mt-6 w-full bg-red-500 hover:bg-red-600 text-white p-3 rounded-lg font-medium text-sm">Clear Entire Checklist</button>
            </section>

            <section class="content-card p-6 md:p-8">
                <h2 class="text-2xl font-semibold mb-5 text-teal-600">Idea Validator</h2>
                <div class="space-y-5">
                    <div>
                        <label for="idea-title" class="block text-sm font-medium text-slate-700 mb-1">Title / Core Idea:</label>
                        <input type="text" id="idea-title" placeholder="E.g., Launching a New Community Initiative" class="input-field w-full p-3 border rounded-lg focus:ring-teal-500 focus:border-teal-500">
                    </div>

                    <h3 class="text-lg font-medium text-slate-700 pt-2">Refinement Questions:</h3>
                    <div class="idea-question">
                        <label for="niche-q1" class="block text-sm font-medium text-slate-600 mb-1">1. Core Theme Alignment:</label>
                        <textarea id="niche-q1" rows="3" placeholder="How does this idea align with your main goals or project themes?" class="input-field w-full p-2 border rounded-lg focus:ring-teal-500 focus:border-teal-500"></textarea>
                    </div>
                    <div class="idea-question">
                        <label for="niche-q2" class="block text-sm font-medium text-slate-600 mb-1">2. Target Audience / User Focus:</label>
                        <textarea id="niche-q2" rows="3" placeholder="Who is this for? What specific problem does it solve or need does it address for them?" class="input-field w-full p-2 border rounded-lg focus:ring-teal-500 focus:border-teal-500"></textarea>
                    </div>
                    <div class="idea-question">
                        <label for="niche-q3" class="block text-sm font-medium text-slate-600 mb-1">3. Unique Angle / Value Proposition:</label>
                        <textarea id="niche-q3" rows="3" placeholder="What's your unique take or key differentiator? How is this different or better?" class="input-field w-full p-2 border rounded-lg focus:ring-teal-500 focus:border-teal-500"></textarea>
                    </div>
                    <div class="idea-question">
                        <label for="niche-q4" class="block text-sm font-medium text-slate-600 mb-1">4. Actionable Outcomes / Key Takeaways:</label>
                        <textarea id="niche-q4" rows="3" placeholder="What 1-3 clear, practical benefits or steps will users gain?" class="input-field w-full p-2 border rounded-lg focus:ring-teal-500 focus:border-teal-500"></textarea>
                    </div>
                     <div class="idea-question">
                        <label for="niche-q5" class="block text-sm font-medium text-slate-600 mb-1">5. Format & Resource Considerations:</label>
                        <textarea id="niche-q5" rows="3" placeholder="What format will this take (e.g., article, video, workshop)? Any specific resources or collaborators needed?" class="input-field w-full p-2 border rounded-lg focus:ring-teal-500 focus:border-teal-500"></textarea>
                    </div>

                    <div class="flex flex-col sm:flex-row space-y-3 sm:space-y-0 sm:space-x-3 mt-6">
                        <button id="save-idea-btn" class="action-btn flex-1 bg-teal-500 hover:bg-teal-600 text-white p-3 rounded-lg font-medium">Save Idea & Notes</button>
                        <button id="clear-idea-btn" class="action-btn flex-1 bg-amber-500 hover:bg-amber-600 text-white p-3 rounded-lg font-medium">Clear Current Fields</button>
                    </div>
                </div>
            </section>
        </div>
        <footer class="text-center mt-12 py-6 border-t border-slate-200">
            <p class="text-sm text-slate-500">&copy; <span id="currentYear"></span> Your Prep Tool. Keep creating!</p>
        </footer>
    </div>

    <div id="message-box" class="message-box"></div>

    <script>
        // --- DOM Element References ---
        // Checklist elements
        const taskForm = document.getElementById('task-form');
        const newTaskInput = document.getElementById('new-task-input');
        const taskList = document.getElementById('task-list');
        const clearChecklistBtn = document.getElementById('clear-checklist-btn');

        // Idea Validator elements
        const ideaTitleInput = document.getElementById('idea-title');
        const nicheQuestionInputs = [ // Array of textarea elements for refinement questions
            document.getElementById('niche-q1'),
            document.getElementById('niche-q2'),
            document.getElementById('niche-q3'),
            document.getElementById('niche-q4'),
            document.getElementById('niche-q5'),
        ];
        const nicheQuestionLabels = [ // Labels for copying content (matches the order of inputs)
            "1. Core Theme Alignment:",
            "2. Target Audience / User Focus:",
            "3. Unique Angle / Value Proposition:",
            "4. Actionable Outcomes / Key Takeaways:",
            "5. Format & Resource Considerations:"
        ];
        const saveIdeaBtn = document.getElementById('save-idea-btn');
        const clearIdeaBtn = document.getElementById('clear-idea-btn');

        // Shared elements
        const messageBox = document.getElementById('message-box'); // For displaying user feedback
        const printPdfBtn = document.getElementById('print-pdf-btn');
        const copyContentBtn = document.getElementById('copy-content-btn');

        // Set current year in footer
        document.getElementById('currentYear').textContent = new Date().getFullYear();

        // --- Message Box Functionality ---
        /**
         * Displays a temporary message to the user.
         * @param {string} message - The message to display.
         * @param {string} type - 'success' or 'error' to style the message box.
         * @param {number} duration - How long the message stays visible (in milliseconds).
         */
        function showMessage(message, type = 'success', duration = 3000) {
            messageBox.textContent = message;
            messageBox.className = `message-box ${type}`; // Apply base and type-specific classes
            void messageBox.offsetWidth; // Trigger a reflow to ensure animation plays
            messageBox.classList.add('show'); // Make it visible and animate in
            setTimeout(() => {
                messageBox.classList.remove('show'); // Hide and animate out
            }, duration);
        }

        // --- Checklist Functionality ---
        let tasks = []; // Array to hold task objects {text: string, completed: boolean, id: number}
        const tasksStorageKey = 'prepTasks_v2'; // Key for localStorage

        /**
         * Loads tasks from localStorage or initializes with default tasks if none are found.
         */
        function loadTasks() {
            const storedTasks = localStorage.getItem(tasksStorageKey);
            if (storedTasks) {
                tasks = JSON.parse(storedTasks);
            } else {
                // Default tasks if local storage is empty
                tasks = [
                    { text: "Brainstorm project topics", completed: false, id: Date.now() + 1 },
                    { text: "Select primary project topic", completed: false, id: Date.now() + 2 }
                ];
            }
            renderTasks(); // Update the UI with loaded tasks
        }

        /**
         * Saves the current tasks array to localStorage.
         */
        function saveTasks() {
            localStorage.setItem(tasksStorageKey, JSON.stringify(tasks));
        }

        /**
         * Renders the list of tasks in the UI.
         * Clears the existing list and rebuilds it based on the 'tasks' array.
         */
        function renderTasks() {
            taskList.innerHTML = ''; // Clear current task list
            if (tasks.length === 0) {
                // Display a message if there are no tasks
                taskList.innerHTML = '<li class="text-slate-500 italic p-3 text-center">Your checklist is empty. Add a task to get started!</li>';
                return;
            }
            tasks.forEach(task => {
                const li = document.createElement('li');
                li.className = `task-item flex items-center justify-between p-3 bg-slate-50 hover:bg-slate-100 rounded-lg shadow-sm transition-all duration-150 ease-in-out ${task.completed ? 'completed' : ''}`;
                li.dataset.id = task.id; // Store task ID for easy access

                // Checkbox and text label
                const checkboxLabel = document.createElement('label');
                checkboxLabel.className = 'flex items-center cursor-pointer';

                const checkboxDiv = document.createElement('div');
                checkboxDiv.className = 'checkbox-icon w-5 h-5 border-2 border-slate-300 rounded-md flex items-center justify-center mr-3 transition-all duration-150';
                if(task.completed) checkboxDiv.classList.add('bg-sky-500', 'border-sky-500');

                const checkmarkSVG = document.createElementNS('http://www.w3.org/2000/svg', 'svg');
                checkmarkSVG.setAttribute('viewBox', '0 0 20 20');
                checkmarkSVG.setAttribute('fill', 'currentColor');
                checkmarkSVG.className = `w-3 h-3 text-white ${task.completed ? 'block' : 'hidden'}`;
                checkmarkSVG.innerHTML = '<path fill-rule="evenodd" d="M16.707 5.293a1 1 0 010 1.414l-8 8a1 1 0 01-1.414 0l-4-4a1 1 0 011.414-1.414L8 12.586l7.293-7.293a1 1 0 011.414 0z" clip-rule="evenodd" />';
                checkboxDiv.appendChild(checkmarkSVG);

                const hiddenCheckbox = document.createElement('input'); // Actual checkbox, visually hidden
                hiddenCheckbox.type = 'checkbox';
                hiddenCheckbox.checked = task.completed;
                hiddenCheckbox.className = 'sr-only'; // Tailwind class for screen-reader only
                hiddenCheckbox.addEventListener('change', toggleTaskComplete);

                const taskTextSpan = document.createElement('span');
                taskTextSpan.textContent = task.text;
                taskTextSpan.className = 'task-text flex-grow text-slate-700';

                checkboxLabel.appendChild(hiddenCheckbox);
                checkboxLabel.appendChild(checkboxDiv);
                checkboxLabel.appendChild(taskTextSpan);

                // Delete button
                const deleteBtn = document.createElement('button');
                deleteBtn.innerHTML = '<svg xmlns="http://www.w3.org/2000/svg" class="h-5 w-5 pointer-events-none" viewBox="0 0 20 20" fill="currentColor"><path fill-rule="evenodd" d="M9 2a1 1 0 00-.894.553L7.382 4H4a1 1 0 000 2v10a2 2 0 002 2h8a2 2 0 002-2V6a1 1 0 100-2h-3.382l-.724-1.447A1 1 0 0011 2H9zM7 8a1 1 0 012 0v6a1 1 0 11-2 0V8zm5-1a1 1 0 00-1 1v6a1 1 0 102 0V8a1 1 0 00-1-1z" clip-rule="evenodd" /></svg>';
                deleteBtn.className = 'delete-btn text-slate-400 hover:text-red-500 font-bold ml-3 p-1 rounded-md hover:bg-red-100 transition-colors duration-150';
                deleteBtn.setAttribute('aria-label', 'Delete task');
                deleteBtn.addEventListener('click', deleteTask);

                li.appendChild(checkboxLabel);
                li.appendChild(deleteBtn);
                taskList.appendChild(li);
            });
        }

        /**
         * Handles the submission of the new task form.
         * Adds a new task to the 'tasks' array and updates the UI.
         * @param {Event} event - The form submission event.
         */
        taskForm.addEventListener('submit', function(event) {
            event.preventDefault(); // Prevent page reload
            const taskText = newTaskInput.value.trim();
            if (taskText === '') {
                showMessage('Task description cannot be empty.', 'error');
                newTaskInput.focus();
                return;
            }
            // Add new task to the beginning of the array
            tasks.unshift({ text: taskText, completed: false, id: Date.now() });
            newTaskInput.value = ''; // Clear input field
            saveTasks();
            renderTasks();
            showMessage('Task added successfully!', 'success');
        });

        /**
         * Toggles the 'completed' status of a task.
         * @param {Event} event - The change event from the task's checkbox.
         */
        function toggleTaskComplete(event) {
            const listItem = event.target.closest('li.task-item'); // Find parent <li>
            if (!listItem) return;
            const taskId = parseInt(listItem.dataset.id);
            const task = tasks.find(t => t.id === taskId);
            if (task) {
                task.completed = !task.completed;
                saveTasks();
                renderTasks(); // Re-render to reflect the change in styling
            }
        }

        /**
         * Deletes a task from the list.
         * @param {Event} event - The click event from a delete button.
         */
        function deleteTask(event) {
            const listItem = event.target.closest('li.task-item');
            if (!listItem) return;
            const taskId = parseInt(listItem.dataset.id);
            const task = tasks.find(t => t.id === taskId);
            const taskTextPreview = task ? `"${task.text.substring(0, 50)}${task.text.length > 50 ? '...' : ''}"` : "this task";

            if (confirm('Are you sure you want to delete the task: ' + taskTextPreview + '?')) {
                tasks = tasks.filter(t => t.id !== taskId); // Remove task from array
                saveTasks();
                renderTasks();
                showMessage('Task deleted.', 'success');
            }
        }

        /**
         * Clears all tasks from the checklist after confirmation.
         */
        clearChecklistBtn.addEventListener('click', function() {
            if (tasks.length === 0) {
                 showMessage('Checklist is already empty.', 'error');
                 return;
            }
            if (confirm('Are you sure you want to delete ALL tasks from the checklist? This action cannot be undone.')) {
                tasks = []; // Empty the tasks array
                saveTasks();
                renderTasks();
                showMessage('All checklist tasks have been cleared.', 'success');
            }
        });

        // --- Idea Validator Functionality ---
        const ideaStorageKey = 'projectIdea_v2'; // Key for localStorage

        /**
         * Loads the saved idea (title and notes) from localStorage.
         */
        function loadIdea() {
            const storedIdea = localStorage.getItem(ideaStorageKey);
            if (storedIdea) {
                const ideaData = JSON.parse(storedIdea);
                ideaTitleInput.value = ideaData.title || '';
                nicheQuestionInputs.forEach((input, index) => {
                    input.value = ideaData.notes[index] || '';
                });
            }
        }

        /**
         * Saves the current idea title and refinement notes to localStorage.
         */
        saveIdeaBtn.addEventListener('click', function() {
            const ideaData = {
                title: ideaTitleInput.value.trim(),
                notes: nicheQuestionInputs.map(input => input.value.trim()) // Get notes from all textareas
            };
            // Check if there's anything to save
            if (!ideaData.title && ideaData.notes.every(note => note === '')) {
                showMessage('Please enter a title or some notes before saving.', 'error');
                ideaTitleInput.focus();
                return;
            }
            localStorage.setItem(ideaStorageKey, JSON.stringify(ideaData));
            showMessage('Idea and notes saved!', 'success');
        });

        /**
         * Clears the input fields for the idea title and refinement questions.
         * Does not clear the data from localStorage until 'Save' is clicked again.
         */
        clearIdeaBtn.addEventListener('click', function() {
            const isAnythingFilled = ideaTitleInput.value.trim() !== '' || nicheQuestionInputs.some(input => input.value.trim() !== '');
            if (!isAnythingFilled) {
                showMessage('Fields are already empty.', 'error');
                return;
            }
            if (confirm('Are you sure you want to clear all current idea fields? This will not affect your saved data until you save again.')) {
                ideaTitleInput.value = '';
                nicheQuestionInputs.forEach(input => input.value = '');
                showMessage('Idea fields cleared. Saved data is intact until you save again.', 'success', 4000);
                ideaTitleInput.focus();
            }
        });

        // --- Export Functionality ---
        /**
         * Triggers the browser's print dialog.
         */
        printPdfBtn.addEventListener('click', function() {
            showMessage('Prepare your print settings (e.g., Save as PDF).', 'success', 4000);
            window.print();
        });

        /**
         * Copies the checklist and idea validator content to the clipboard.
         * Tries to copy as HTML first, then falls back to plain text.
         */
        copyContentBtn.addEventListener('click', function() {
            let contentToCopy = "<h1>Project Power Prep Data</h1>\\n\\n";

            // Checklist Content
            contentToCopy += "<h2>Pre-Launch Checklist</h2>\\n";
            if (tasks.length > 0) {
                contentToCopy += "<ul>\\n";
                tasks.forEach(task => {
                    // Mark tasks as [DONE] or [TODO] for clarity
                    contentToCopy += `    <li>${task.completed ? '[DONE] ' : '[TODO] '} ${task.text}</li>\\n`;
                });
                contentToCopy += "</ul>\\n\\n";
            } else {
                contentToCopy += "<p>No tasks in the checklist.</p>\\n\\n";
            }

            // Idea Validator Content
            contentToCopy += "<h2>Idea Validator</h2>\\n";
            const ideaTitle = ideaTitleInput.value.trim();
            contentToCopy += `<h3>Title / Core Idea:</h3>\\n<p>${ideaTitle || 'Not specified'}</p>\\n\\n`;

            contentToCopy += "<h3>Refinement Questions:</h3>\\n";
            nicheQuestionInputs.forEach((input, index) => {
                contentToCopy += `<h4>${nicheQuestionLabels[index]}</h4>\\n`; // Use predefined labels
                contentToCopy += `<p>${input.value.trim() || 'No answer provided.'}</p>\\n`;
            });

            // Attempt to copy as HTML (for rich text editors that support it)
            try {
                if (navigator.clipboard && navigator.clipboard.write) {
                    const blob = new Blob([contentToCopy], { type: 'text/html' });
                    const clipboardItem = new ClipboardItem({ 'text/html': blob });
                    navigator.clipboard.write([clipboardItem]).then(() => {
                        showMessage('Content copied to clipboard as HTML!', 'success');
                    }).catch(err => {
                        console.warn('HTML clipboard copy failed, falling back to plain text.', err);
                        copyAsPlainText(contentToCopy); // Fallback to plain text
                    });
                } else {
                    throw new Error('Clipboard API (write) not supported.');
                }
            } catch (e) {
                 // Fallback for older browsers or if ClipboardItem/write is not supported
                console.warn('ClipboardItem or advanced clipboard API not supported, falling back to plain text.', e);
                copyAsPlainText(contentToCopy);
            }
        });

        /**
         * Fallback function to copy content as plain text.
         * Converts simple HTML to a basic plain text format.
         * @param {string} htmlContent - The HTML content to convert and copy.
         */
        function copyAsPlainText(htmlContent) {
            // Basic conversion from simple HTML to plain text
            const plainText = htmlContent
                .replace(/<h1>(.*?)<\\/h1>/gi, "\\n## $1\\n")
                .replace(/<h2>(.*?)<\\/h2>/gi, "\\n### $1\\n")
                .replace(/<h3>(.*?)<\\/h3>/gi, "\\n#### $1\\n")
                .replace(/<h4>(.*?)<\\/h4>/gi, "\\n##### $1\\n")
                .replace(/<ul>/gi, "")
                .replace(/<\\/ul>/gi, "")
                .replace(/<li>(.*?)<\\/li>/gi, "- $1\\n")
                .replace(/<p>(.*?)<\\/p>/gi, "$1\\n")
                .replace(/<br\\s*\\/?>/gi, "\\n")
                .replace(/&nbsp;/g, " ")
                .replace(/&amp;/g, "&")
                .replace(/&lt;/g, "<")
                .replace(/&gt;/g, ">")
                .replace(/&quot;/g, "\\"")
                .replace(/&#39;/g, "'")
                .replace(/(\\r\\n|\\n|\\r){2,}/g, '\\n\\n') // Normalize multiple newlines
                .trim();

            if (navigator.clipboard && navigator.clipboard.writeText) {
                navigator.clipboard.writeText(plainText).then(() => {
                    showMessage('Content copied to clipboard as plain text!', 'success');
                }).catch(err => {
                    showMessage('Failed to copy content. Please try selecting and copying manually.', 'error');
                    console.error('Failed to copy plain text: ', err);
                });
            } else {
                // Very basic fallback if writeText is also not available (e.g. insecure context)
                // This would require manual copy from a textarea, but for now, just log.
                showMessage('Automatic copy not fully supported. Please copy manually if needed.', 'error');
                console.error('navigator.clipboard.writeText not available.');
            }
        }

        // --- Initial Load ---
        /**
         * Event listener for when the DOM is fully loaded.
         * Calls functions to load data from localStorage.
         */
        document.addEventListener('DOMContentLoaded', () => {
            loadTasks();  // Load checklist tasks
            loadIdea();   // Load idea validator data
        });

    </script>
</body>
</html>
"""

# Content for instructions.txt
instructions_content = """
Instructions for Project Power Prep Tool (pod_prep.html)
========================================================

Thank you for downloading the Project Power Prep Tool! This tool is designed to help you streamline your project creation process, from initial idea validation to pre-launch checks.

What's Inside the ZIP:
----------------------
1.  pod_prep.html: This is the main interactive tool. Open it in any modern web browser (like Chrome, Firefox, Safari, or Edge).
2.  instructions.txt: This file.

How to Use pod_prep.html:
-------------------------
1.  Open `pod_prep.html` in your web browser.
    * You don't need an internet connection to use the tool after it's opened once (as it uses CDN for Tailwind CSS, which might be cached by your browser). For fully offline use, you would need to download Tailwind CSS and link it locally.

2.  Pre-Launch Checklist:
    * Add Tasks: Type a task into the input field under "Pre-Launch Checklist" (e.g., "Finalize script," "Record audio") and click the "Add" button.
    * Mark Complete: Click the checkbox next to a task to mark it as done. The task will be visually updated (strikethrough).
    * Delete Task: Click the "✕" (delete icon) next to a task to remove it from the list. You'll be asked to confirm.
    * Clear All: Use the "Clear Entire Checklist" button to remove all tasks. A confirmation will be required.
    * Data Storage: Your checklist tasks are automatically saved in your browser's local storage. This means if you close the page and reopen it in the same browser, your tasks will still be there. This data is stored *only* in your browser, not sent anywhere.

3.  Idea Validator:
    * Enter Idea: Type your project title or core concept into the "Title / Core Idea" field.
    * Refine with Questions: Answer the five refinement questions to help you flesh out your idea, consider your audience, your unique angle, actionable outcomes, and necessary resources.
    * Save Notes: Click the "Save Idea & Notes" button. Your title and the answers to the questions will be saved in your browser's local storage.
    * Clear Fields: Click "Clear Current Fields" to empty the title and all question textareas. This *does not* delete your saved data immediately; it only clears what's currently visible. If you want to clear saved data, clear the fields and then click "Save Idea & Notes" with the empty fields.
    * Data Persistence: Similar to the checklist, your idea title and notes are saved in your browser's local storage.

4.  Exporting Your Data:
    * Prepare for Print / PDF: Click this button to open your browser's print dialog. You can then choose to print the content or, more commonly, "Save as PDF" to create a PDF document of your checklist and idea notes. The page is styled for a cleaner print output.
    * Copy Content for Docs: Click this button to copy a formatted version of your checklist and idea notes to your clipboard. You can then paste this into a word processor, note-taking app, or email. It attempts to copy as HTML for rich formatting, falling back to plain text.

Tips for Effective Use:
-----------------------
* Consistency: Use this tool for each new project or significant piece of content you plan to create.
* Local Storage: Remember that data is stored locally in your browser. If you clear your browser's cache/data for this site or use a different browser/device, the saved information will not be available there.
* Backup: For very important information, consider using the "Copy Content for Docs" or "Print to PDF" feature regularly to create backups outside of your browser's local storage.

Troubleshooting:
----------------
* If the tool doesn't look right (e.g., styling is off): Ensure you have an internet connection the first time you open it so it can load Tailwind CSS. If you continue to have issues, try a different browser.
* If data isn't saving: Ensure your browser isn't set to block local storage for local files or that you're not in a private/incognito mode that clears data on exit.

We hope this tool helps you stay organized and bring your creative projects to life!
"""

# Name of the ZIP file to be created
zip_filename = "podcast_prep_tool.zip"

# Create a new ZIP file
# Using 'w' mode, which creates a new file or truncates an existing one.
try:
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add pod_prep.html to the ZIP file
        # The arcname parameter specifies the name and path of the file within the archive.
        zipf.writestr("pod_prep.html", html_content)
        print(f"Added pod_prep.html to {zip_filename}")

        # Add instructions.txt to the ZIP file
        zipf.writestr("instructions.txt", instructions_content)
        print(f"Added instructions.txt to {zip_filename}")

    print(f"\nSuccessfully created {zip_filename} in the current directory.")
    print("This ZIP file contains 'pod_prep.html' and 'instructions.txt'.")

except IOError as e:
    print(f"An I/O error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Added pod_prep.html to podcast_prep_tool.zip
Added instructions.txt to podcast_prep_tool.zip

Successfully created podcast_prep_tool.zip in the current directory.
This ZIP file contains 'pod_prep.html' and 'instructions.txt'.
